# Part 01 - Exercise 3 (n-gram language models)

## Organizing our Data

At first, we downloaded via *nltk* package a valuable corpus such as 'reuters'. Moreover, we downloaded the method tokenization 'punkt' via nltk package.
We splitted our data into training, development and test set and we transformed any rare word (freq<=10) or out-of-vocabulary word to the special token 'UNK'.
As we can see from the printed console, a lot of words transformed into the special token 'UNK' in order to be able to handle the unknown words better.

In [136]:
# If you running for first time uncomment the following 3 lines iot download the corpus
# import nltk
# nltk.download('reuters')
# nltk.download('punkt')
from nltk.corpus import reuters
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from collections import Counter
from nltk.util import ngrams
import math
from more_itertools import windowed


# Load the 'reuters' corpus
sentences = reuters.sents()
# Splitting data into Training, Development and Test set
train_sents, test_sents = train_test_split(reuters.sents(), test_size=0.1, random_state=42)
dev_sents, test_sents = train_test_split(test_sents, test_size=0.5, random_state=42)


# Transform the train sentences into words
train_words = [word for sentence in train_sents for word in sentence]
freq_dist_train = FreqDist(train_words)

# Replace rare words in train set
cleaned_train_sentences = []
for sentence in train_sents:
    cleaned_train_sentence = [word if freq_dist_train[word] > 5 else '<UNK>' for word in sentence]
    cleaned_train_sentences.append(cleaned_train_sentence)

# Build our vocab based on training set
vocab = set([word for sentence in cleaned_train_sentences for word in sentence])

print(f'The total size of our vocabulary is: {len(vocab)}')


print('Example sentence of our training set:')
print('|'.join(cleaned_train_sentences[0]))

# Transform the development sentences into words
dev_words = [word for sentence in dev_sents for word in sentence]
freq_dist_dev = FreqDist(dev_words)

# Replace rare words or Out-of-Vocabulary words in dev set
cleaned_dev_sentences = []
for sentence in dev_sents:
    cleaned_dev_sentence = [word if word in vocab else '<UNK>' for word in sentence]
    cleaned_dev_sentences.append(cleaned_dev_sentence)

print('Example sentence of our development set:')
print('|'.join(cleaned_dev_sentences[0]))

# Transform the test sentences into words
test_words = [word for sentence in test_sents for word in sentence]
freq_dist_test = FreqDist(test_words)

# Replace rare words or Out-of-Vocabulary words in test set
cleaned_test_sentences = []
for sentence in test_sents:
    cleaned_test_sentence = [word if word in vocab else '<UNK>' for word in sentence]
    cleaned_test_sentences.append(cleaned_test_sentence)

print('Example sentence of our test set:')
print('|'.join(cleaned_test_sentences[0]))

The total size of our vocabulary is: 12038
Example sentence of our training set:
It|has|appeared|to|traders|for|some|years|that|the|EC|'|s|export|policy|is|<UNK>|responsive|to|changing|patterns|of|demand|,|it|says|.
Example sentence of our development set:
RIO|DE|<UNK>|<UNK>|STRIKE|Rio|de|Janeiro|'|s|3|,|500|<UNK>|went|on|strike|for|an|indefinite|period|today|to|demand|wage|increases|,|a|spokesman|for|the|<UNK>|said|.
Example sentence of our test set:
The|foreign|currency|purchased|was|used|by|the|Treasury|to|repay|foreign|debt|and|did|not|affect|the|Bank|'|s|foreign|exchange|reserves|.


## i) Build our unigram, bigram & trigram model

In the following block of code we used Counter() in order to cosntruct our ngram models (unigram, bigram, trigram) and simultaneously to count the frequency of each word. In the printed console we can observe the most common words for each ngram model.

In [103]:
from pprint import pprint

# Build unigram, bigram and trigram counters for our training set
unigram_counter = Counter()
bigram_counter = Counter()
trigram_counter = Counter()

for sent in cleaned_train_sentences:

    unigram_counter.update([gram for gram in ngrams(sent, 1, pad_left=True, pad_right=True,
                                                    left_pad_symbol='<s>', right_pad_symbol='<e>')])
    bigram_counter.update([gram for gram in ngrams(sent, 2, pad_left=True, pad_right=True,
                                                       left_pad_symbol='<s>', right_pad_symbol='<e>')])
    trigram_counter.update([gram for gram in ngrams(sent, 3, pad_left=True, pad_right=True,
                                                        left_pad_symbol='<s>', right_pad_symbol='<e>')])

print('10 most common words for unigram counter:')
pprint(unigram_counter.most_common(10))
print('10 most common words for bigram counter:')
pprint(bigram_counter.most_common(10))
print('10 most common words for trigram counter:')
pprint(trigram_counter.most_common(10))

10 most common words for unigram counter:
[(('.',), 84992),
 ((',',), 64910),
 (('the',), 52391),
 (('<UNK>',), 52158),
 (('of',), 32300),
 (('to',), 30666),
 (('in',), 23750),
 (('said',), 22668),
 (('and',), 22618),
 (('a',), 21061)]
10 most common words for bigram counter:
[(('.', '<e>'), 43915),
 ((',', '000'), 9173),
 (("'", 's'), 8336),
 (('<s>', 'The'), 7992),
 (('lt', ';'), 7845),
 (('&', 'lt'), 7842),
 (('said', '.'), 7119),
 (('<s>', '<UNK>'), 6563),
 (('of', 'the'), 6131),
 (('in', 'the'), 5792)]
10 most common words for trigram counter:
[(('.', '<e>', '<e>'), 43915),
 (('<s>', '<s>', 'The'), 7992),
 (('&', 'lt', ';'), 7841),
 (('said', '.', '<e>'), 7118),
 (('<s>', '<s>', '<UNK>'), 6563),
 (('lt', ';', '<UNK>'), 5153),
 (('U', '.', 'S'), 5100),
 (('.', 'S', '.'), 4780),
 ((';', '<UNK>', '>'), 3299),
 (('<s>', '<s>', '"'), 3237)]


sentence1sentence3
sentence2sentence4


## Calculation of bigram and trigram probabilities via Laplace smoothing

In the following block of code we constructed a function which is responsible for calculating the probability of a ngram (bigram or trigram) model.
We used Laplace smoothing for this purpose. We also added the special tokens in order to include them in the size of the vocabulary.

In [104]:
# Define the hyperparameter alpha. Fine-tuning on the development set
alpha = 0.1

# Sum the tokens for the whole corpus (training, dev & test sets)
tokens = [token for sent in sentences for token in sent]
# Calculate vocabulary size (including any special tokens)
special_tokens = ['<s>', '<e>']
vocab_size = len(vocab) + len(special_tokens)
print(f'The total size of our vocabulary size based on the training set and special tokens is: {vocab_size}')


def calc_ngram_proba(ngram_counter, ngram_minus_one_counter, ngram, alpha, vocab_size):
    """
    Calculate ngram probability with Laplace smoothing
    :param bigram_counter: Counter which the key is a tuple of ngram and value its frequency
    :param gram_counter: Counter which the key is a tuple of n-1gram and value its frequency
    :param ngram: tuple
    :param alpha: float hyperparameter for Laplace smoothing
    :param vocab_size: int value which defines the whole size of the corpus
    :return: float probability of the ngram inside the corpus
    """
    ngram_count = ngram_counter[ngram]
    context = ngram[:-1]
    if context == ('<s>', '<s>',) or context == ('<s>',):
        ngram_minus_one_count = len(cleaned_train_sentences)
    else:
        ngram_minus_one_count = ngram_minus_one_counter[context]
    if ngram_count>ngram_minus_one_count:
        print(f'The following ngram occurs an error in the counter: {ngram}')
    ngram_prob = (ngram_count + alpha) / (ngram_minus_one_count + (alpha * vocab_size))
    return math.log2(ngram_prob)


The total size of our vocabulary size based on the training set and special tokens is: 12040


## ii) Calculation of probabilities, Cross-Entropy and Perplexity for our bigram model (Laplace smoothing)

In [105]:
# Calculate bigram probability and Cross-Entropy of sentences in the test set
total_log_proba_bigram = 0.0
for sent in cleaned_test_sentences:
    # Pad the sentence with '<s>' and '<e>' tokens
    padded_sent = ['<s>'] + sent + ['<e>']

    # Iterate over the bigrams of the sentence
    for first_token, second_token in windowed(padded_sent, 2):
        # if first_token == '<s>': # Avoid calculating that because unigram counter does not have counts for <s>
        #     pass
        # else:
        bigram = (first_token, second_token)
        bigram_prob = calc_ngram_proba(bigram_counter, unigram_counter, bigram, alpha, vocab_size)
        total_log_proba_bigram += bigram_prob

# Calculation of total tokens for test set, including only 'end' token for each sentence
num_tokens = sum(len(sent) + 2 for sent in cleaned_test_sentences)

cross_entropy_bigram = - total_log_proba_bigram / num_tokens
print(f"The total Cross-Entropy of bigram model for our Test set is: {cross_entropy_bigram:.3f}")

# Calculation of the perplexity of bigram model for the test set

bigram_perplexity = 2 ** (cross_entropy_bigram)
print(f"Perplexity of bigram model for Test Set: {bigram_perplexity:.3f}")

The total Cross-Entropy of bigram model for our Test set is: 7.155
Perplexity of bigram model for Test Set: 142.546


## Calculation of probabilities, Cross-Entropy and Perplexity for our trigram model

In [106]:
# Calculate trigram probability and Cross-Entropy of sentences in the test set
total_log_proba_trigram = 0.0
for sent in cleaned_test_sentences:
    # Pad the sentence with '<s>' and '<e>' tokens
    padded_sent = ['<s>'] + ['<s>'] + sent + ['<e>']

    # Iterate over the bigrams of the sentence
    for first_token, second_token, third_token in windowed(padded_sent, 3):
        if first_token == '<s>' and second_token == '<s>': # Avoid calculating that because bigram counter does not have counts for <s>, <s>
            pass
        else:
            trigram = (first_token, second_token, third_token)
            trigram_prob = calc_ngram_proba(trigram_counter, bigram_counter, trigram, alpha, vocab_size)
            total_log_proba_trigram += trigram_prob

cross_entropy_trigram = - total_log_proba_trigram / num_tokens
print(f"The total Cross-Entropy of trigram model for our Test set is: {cross_entropy_trigram: .3f}")

# Calculation of the perplexity of bigram model for the test set
trigram_perplexity = 2 ** (cross_entropy_trigram)
print(f"Perplexity of trigram model for Test Set: {trigram_perplexity:.3f}")

The total Cross-Entropy of trigram model for our Test set is:  8.801
Perplexity of trigram model for Test Set: 445.916


## Calculation of bigram and trigram probabilities via Improved Kneser-Ney smoothing

In the following block of code we constructed a function which is responsible for calculating the probability of a ngram (bigram or trigram) model.
In the following block of code we used Kneser-Ney smoothing which is more challenging and efficient. We generalized the purpose of our function in order to calculate either for bigram or trigram models.

In [125]:
def calc_kneser_ney_proba(ngram_counter, ngram_minus_one_counter, continuation_counts, ngram, delta, prefixes_counter):
    """
    Calculate ngram probability with simplified Kneser-Ney smoothing for bigrams or trigrams
    :param ngram_counter: Counter for ngrams (bigrams or trigrams)
    :param ngram_minus_one_counter: Counter for n-1 grams
    :param continuation_counts: Counter for continuation counts
    :param total_continuations: Total number of unique continuations
    :param ngram: tuple representing the ngram (bigram or trigram)
    :param delta: discount value
    :param prefixes_counter: Counter for prefixes of ngram
    :return: float probability of the ngram
    """
    # print(ngram)
    ngram_count = ngram_counter[ngram]
    context = ngram[:-1]
    if context == ('<s>', '<s>',) or context == ('<s>',):
        ngram_minus_one_count = len(cleaned_train_sentences)
    else:
        ngram_minus_one_count = ngram_minus_one_counter[context]

    adjusted_count = max(ngram_count - delta, 0)
    epsilon = 1e-10

    # For bigrams, use the second token for continuation, for trigrams use the third token
    continuation_token = ngram[-1]

    # Calculate our interpolation weight
    continuation_prob = continuation_counts[continuation_token] / len(ngram_counter)
    if continuation_prob>=1 or continuation_prob==0:
        print(f'The continuation probability is: {continuation_prob:.3f}')
    alpha_weight = (delta * prefixes_counter[(context)] + epsilon) / (ngram_minus_one_count + epsilon)
    kn_probability = adjusted_count / (ngram_minus_one_count + epsilon) + alpha_weight * continuation_prob
    if kn_probability==0 or kn_probability>=1:
        print(f'Error occured with ngram: {ngram}. Probability more than 1 or 0.')
    return kn_probability


## Use of Kneser-Ney smoothing technique and Calculation of probabilities, Cross-Entropy and Perplexity - Refined Version

We implemented Kneser-Ney smoothing ....

In [109]:
from tqdm import tqdm

# Calculate continuation counts
continuation_counts_bi = Counter([bigram[1] for bigram in bigram_counter])


# Convert list of n-grams to a list of tuples
ngram_tuples_bi = [tuple(ng) for ng in bigram_counter]

# Create a Counter for the prefixes
prefixes_counter_bi = Counter(ng[:-1] for ng in ngram_tuples_bi)

total_log_proba_bigram_kn = 0.0
delta = 0.5
with tqdm(total=256000) as pbar:  # Check our time and iters remaining!
    for sent in cleaned_test_sentences:
        padded_sent = ['<s>'] + sent + ['<e>']

        for first_token, second_token in windowed(padded_sent, 2):
            bigram = (first_token, second_token)
            bigram_prob = calc_kneser_ney_proba(bigram_counter, unigram_counter, continuation_counts_bi,
                                                    bigram, delta, prefixes_counter_bi)
            total_log_proba_bigram_kn += math.log2(bigram_prob)
            pbar.update(1)  # Update the progress bar

cross_entropy_bigram_kn = - total_log_proba_bigram_kn / num_tokens
print(f"The total Cross-Entropy of bigram model via Kneser-Ney smoothing for our Test set is: {cross_entropy_bigram_kn: .3f}")

# Calculation of the perplexity of bigram model for the test set via Kneser-Ney smoothing
bigram_perplexity_kn = 2 ** (cross_entropy_bigram_kn)
print(f"Perplexity of bigram model for Test Set: {bigram_perplexity_kn:.3f}")

 35%|███▍      | 89223/256000 [00:00<00:00, 373374.11it/s]

The total Cross-Entropy of bigram model via Kneser-Ney smoothing for our Test set is:  6.185
Perplexity of bigram model for Test Set: 72.765


## Calculation of probabilities, Cross-Entropy and Perplexity for our trigram model (Kneser-Ney smoothing)

In [122]:
# Calculate continuation counts
continuation_counts_tri = Counter([trigram[2] for trigram in trigram_counter])

total_log_proba_trigram_kn = 0.0
delta = 0.5

# Convert list of n-grams to a list of tuples
ngram_tuples_tri = [tuple(ng) for ng in trigram_counter]

# Create a Counter for the prefixes
prefixes_counter_tri = Counter(ng[:-1] for ng in ngram_tuples_tri)

with tqdm(total=90000) as pbar:  # Check our time and iters remaining!
    for sent in cleaned_test_sentences:
        padded_sent = ['<s>'] + ['<s>'] + sent + ['<e>']

        for first_token, second_token, third_token in windowed(padded_sent, 3):
            trigram = (first_token, second_token, third_token)
            trigram_prob = calc_kneser_ney_proba(trigram_counter, bigram_counter, continuation_counts_tri,
                                                     trigram, delta, prefixes_counter_tri)
            if trigram_prob>=1:
                print(trigram_prob)
            total_log_proba_trigram_kn += math.log2(trigram_prob)
            pbar.update(1)  # Update the progress bar

cross_entropy_trigram_kn = - total_log_proba_trigram_kn / num_tokens
print(f"The total Cross-Entropy of trigram model for our Test set is: {cross_entropy_trigram_kn: .3f}")

# Calculation of the perplexity of bigram model for the test set
trigram_perplexity_kn = 2 ** (cross_entropy_trigram_kn)
print(f"Perplexity of trigram model for Test Set: {trigram_perplexity_kn:.3f}")

 99%|█████████▉| 89223/90000 [00:00<00:00, 275384.59it/s]

The total Cross-Entropy of trigram model for our Test set is:  6.086
Perplexity of trigram model for Test Set: 67.945


## Autocomplete an incomplete sentence

auto......

In [111]:
def generate_candidates(state, ngram_counter, model_name):
    """

    :param state:
    :param ngram_counter:
    :param model_name:
    :return:
    """
    ngram_width = 1
    if model_name == 'trigram':
      ngram_width = 2
    prev_words = tuple(state[-ngram_width:])

    # Find candidates words
    next_words = [prev_words_tuple[-1] for prev_words_tuple in ngram_counter if prev_words == tuple(prev_words_tuple[:-1])]
    if next_words == []:
        if model_name == 'trigram':
            if prev_words[0] and prev_words[1] not in vocab:
                prev_words = ('<UNK>', '<UNK>',)
            elif prev_words[0] not in vocab:
                prev_words = ('<UNK>', prev_words[1],)
            else:
                prev_words = (prev_words[0], '<UNK>',)
            if prev_words in prefixes_counter_tri:
                next_words = [prev_words_tuple[-1] for prev_words_tuple in ngram_counter if prev_words == tuple(prev_words_tuple[:-1])]
                output = [state + [next_word] for next_word in next_words]
            else:
                 return generate_candidates(state, bigram_counter, 'bigram')
        else:
            prev_words = ('<UNK>',)
            next_words = [prev_words_tuple[-1] for prev_words_tuple in ngram_counter if prev_words == tuple(prev_words_tuple[:-1])]
            output = [state + [next_word] for next_word in next_words]
    else:
        output = [state + [next_word] for next_word in next_words]
    return output

In [112]:
def score(state, vocab_size, ngram_counter, ngram_minus_one_counter, prefixes_count,model_name='trigram', dist=0, l1=1, l2=0,
          calculate_ngram_probability_fn=calc_kneser_ney_proba):
    """

    :param state:
    :param vocab_size:
    :param alpha:
    :param ngram_counter:
    :param ngram_minus_one_counter:
    :param model_name:
    :param dist:
    :param l1:
    :param l2:
    :param calculate_ngram_probability_fn:
    :return:
    """
    if model_name == 'trigram':
      ngram_width = 3
      # Re-assign the correct continuation counts for the respective model
      con_counts = continuation_counts_tri
    else:
       ngram_width = 2

      # Re-assign the correct continuation counts for the respective model
       con_counts = continuation_counts_bi
    probability = 0
    prev_words = tuple(state[-ngram_width:])
    # for i in range(ngram_width, len(state)):
    #     prev_words = tuple(state[i-ngram_width:i+1])
    probability += math.log2(l1*calc_kneser_ney_proba(ngram_counter, ngram_minus_one_counter, continuation_counts_bi,
                                            prev_words, 0, prefixes_count) + l2 * 1 / (dist + 1))
    return probability

In [113]:
def beam_search_sequence(initial_state, max_depth, beam_width, vocab_size, ngram_counter,
                         ngram_minus_one_counter, generate_candidates_fn, score_fn):
    """

    :param initial_state:
    :param max_depth:
    :param beam_width:
    :param vocab_size:
    :param alpha:
    :param ngram_counter:
    :param ngram_minus_one_counter:
    :param generate_candidates_fn:
    :param score_fn:
    :return:
    """
    candidates = [(initial_state, 0)]

    for depth in range(max_depth):
        new_candidates = []
        for candidate, prob in candidates:
            for next_state in generate_candidates_fn(candidate, bigram_counter, 'bigram'):

                new_prob = prob + score_fn(next_state, vocab_size, ngram_counter, ngram_minus_one_counter,
                                           prefixes_counter_bi,'bigram')

                if next_state[-1] == '<UNK>' :
                    pass
                else:
                    new_candidates.append((next_state, new_prob))

        new_candidates = sorted(new_candidates, key=lambda x: x[1], reverse=True)

        candidates = new_candidates[:beam_width]
        print(candidates)
    best_sequence, best_prob = max(candidates, key=lambda x: x[1])
    return best_sequence

In [126]:
# Build the vocab
vocab = [word[0] for word in unigram_counter]
print(f'Number of tokens in train set: {len(vocab)}')

test_sentence = "The report"
initial_state = test_sentence.split(' ')[-1:]
max_depth = 10
beam_width = 5
best_sequence = beam_search_sequence(initial_state, max_depth, beam_width,len(vocab),bigram_counter,unigram_counter, generate_candidates, score)

print(f'The 10 best words for autocomplete the sentence "{test_sentence}" is: ')
print(test_sentence, ' '.join(best_sequence[1:]))  # Excluding the "<start>" token


Number of tokens in train set: 12038
[(['report', 'said'], -2.668103736021588), (['report', '.'], -2.9961579337057733), (['report', ','], -3.3219280948875167), (['report', 'on'], -3.9657842846622637), (['report', 'a'], -4.158429362604659)]
[(['report', '.', '<e>'], -3.9487711817484517), (['report', 'said', '.'], -4.339014337216016), (['report', 'said', 'it'], -5.334360804545055), (['report', 'said', 'the'], -5.486319009456026), (['report', 'on', 'the'], -6.1389192626523545)]
[(['report', 'said', '.', '<e>'], -5.291627585258694), (['report', 'said', '.', 'S'], -8.367540035397306), (['report', 'said', 'it', 'has'], -8.775556500622955), (['report', 'said', 'it', 'said'], -9.020157417425974), (['report', 'said', 'it', 'is'], -9.111864558586156)]
[(['report', 'said', '.', 'S', '.'], -8.596795929486536), (['report', 'said', 'it', 'said', '.'], -10.691068018620403), (['report', 'said', '.', '<e>', ','], -10.953088460970571), (['report', 'said', '.', '<e>', '.'], -11.227483688799554), (['repor

4912

In [ ]:
bigram_counter[("that","they")]

87

## Autocomplete with a trigram model

In [57]:
def beam_search_decode(initial_state, max_depth, beam_width, generate_candidates_fn, score_fn):
    """

    :param initial_state:
    :param max_depth:
    :param beam_width:
    :param generate_candidates_fn:
    :param score_fn:
    :return:
    """
    candidates = [(initial_state, 0)]

    for depth in range(max_depth):
        new_candidates = []
        for candidate, prob in candidates:
            for next_state in generate_candidates_fn(candidate, trigram_counter, 'trigram'):
                new_prob = prob + score_fn(next_state, len(vocab), trigram_counter, bigram_counter,
                                           prefixes_counter_tri,'trigram')


                if next_state[-1] == '<UNK>' :
                    pass
                else:
                    new_candidates.append((next_state, new_prob))


        new_candidates = sorted(new_candidates, key=lambda x: x[1], reverse=True)
        candidates = new_candidates[:beam_width]

    best_sequence, best_prob = max(candidates, key=lambda x: x[1])
    return best_sequence


test_sentence = "The report says that"
initial_state = test_sentence.split(' ')[-2:]
max_depth = 20
beam_width = 3
best_sequence = beam_search_decode(initial_state, max_depth, beam_width, generate_candidates, score)
print(' '.join(best_sequence))



says that later in the U . S . Agriculture Department said . <e> <e> and , in a statement . <e>


In [ ]:
prefixes_counter_tri['of', 'whom',]

In [ ]:
trigram_counter['the', '<UNK>','is']

## iv) Develop a context-aware spelling corrector

In [30]:
# Leveinstein Destance with transposition.
def damerau_levenshtein_distance(s1, s2):
    """
    Calculate the Damerau–Levenshtein distance between two strings.

    Parameters:
    - s1: first string
    - s2: second string

    Returns:
    - Damerau Levenshtein distance
    """
    len_s1 = len(s1)
    len_s2 = len(s2)
    d = [[0] * (len_s2 + 1) for _ in range(len_s1 + 1)]

    for i in range(len_s1 + 1):
        d[i][0] = i
    for j in range(len_s2 + 1):
        d[0][j] = j

    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            cost = 0 if s1[i - 1] == s2[j - 1] else 1
            d[i][j] = min(
                d[i - 1][j] + 1,  # deletion
                d[i][j - 1] + 1,  # insertion
                d[i - 1][j - 1] + cost,  # substitution
            )
            if i > 1 and j > 1 and s1[i - 1] == s2[j - 2] and s1[i - 2] == s2[j - 1]:
                d[i][j] = min(d[i][j], d[i - 2][j - 2] + cost)  # transposition

    return d[len_s1][len_s2]


In [94]:
# Take a word and the vocab and produce candidates/
def generate_candidate_with_distance(state, word, word_list, max_candidates=20, distance_fn=damerau_levenshtein_distance):
    """
    Generate candidate words for a misspelled word, using between words distance.

    Parameters:
    - state: The current state.
    - word: The misspelled word.
    - word_list: List of words to search for candidates.
    - max_candidates: Maximum number of candidates
    - distance_fn: Distance function. Deffault damerau_levenshtein_distance

    Returns:
    - A list of candidate words.
    """
    candidates = []

    for candidate in word_list:
        distance = distance_fn(word, candidate)

        candidates.append((candidate, distance))

    # Sort candidates by Distance distance in ascending order
    candidates.sort(key=lambda x: x[1])
    next_words = candidates[:max_candidates]

    # Return next word and distance
    return [(state + [next_word[0]], next_word[1]) for next_word in next_words]

# Example usage
misspelled_word = "apqlicathoo"
initial_state = ['<s>','<s>']
candidates = generate_candidate_with_distance(initial_state,misspelled_word, vocab, 10)
print(f"Candidate words for '{misspelled_word}' is: ")
for cand in candidates:
  print(cand[0][2], end=" | ")

Candidate words for 'apqlicathoo' is: 
application | applications | allocation | acquisition | appreciation | liquidation | association | location | indicator | utilization | 

In [95]:
from nltk.tokenize import word_tokenize

def beam_search_spelling(sentence, beam_width, l1, l2, generate_candidates_fn, score_fn):
    """
    Spelling correction with contect awereness using beam search

    Parameters:
    - sentence: The sentence we try to correct
    - beam_width: The width of beam search algorithm.
    - generate_candidates_fn: function that generates candidate words
    - score_fn: Function that calculates the log probability

    Returns:
    - The most probable sequence corrected.
    """
    print(l1,l2)
    initial_state = ['<s>','<s>']
    candidates = [(initial_state, 0)]
    # sentence = word_tokenize(sentence)
    max_depth = len(sentence)
    for depth in range(max_depth):
        new_candidates = []
        for candidate, prob in candidates:
            for next_state, dist in generate_candidates_fn(candidate, sentence[depth],vocab):

                # Prob we add the previous prob, the prob of the next state and the inverse of the distance
                new_prob = prob + score_fn(next_state,len(vocab), trigram_counter, bigram_counter, prefixes_counter_tri,'trigram', dist, l1, l2)

                new_candidates.append((next_state, new_prob))


        new_candidates = sorted(new_candidates, key=lambda x: x[1], reverse=True)

        candidates = new_candidates[:beam_width]
        print(candidates)
    best_sequence, best_prob = max(candidates, key=lambda x: x[1])
    return best_sequence[2:]


test_sentence = word_tokenize("The deparmt office reprts ")
beam_width = 5
best_sequence = beam_search_spelling(test_sentence, beam_width, 0.9, 0.1, generate_candidate_with_distance, score)
print("The corrected sentence is", end=': ')
print(' '.join(best_sequence))  # Excluding the "<start>" token

0.9 0.1
[(['<s>', '<s>', 'The'], -2.0296607112181833), (['<s>', '<s>', 'He'], -4.055983688419189), (['<s>', '<s>', 'They'], -4.123348515338661), (['<s>', '<s>', 'She'], -4.2943940566748875), (['<s>', '<s>', 'the'], -4.317865751845653)]
[(['<s>', '<s>', 'The', 'department'], -6.924611532843555), (['<s>', '<s>', 'The', 'report'], -6.962695392614918), (['<s>', '<s>', 'The', 'deposit'], -7.334842842990012), (['<s>', '<s>', 'The', 'part'], -7.3515888061055445), (['<s>', '<s>', 'The', 'separate'], -7.351588806105546)]
[(['<s>', '<s>', 'The', 'department', 'office'], -10.246539627730915), (['<s>', '<s>', 'The', 'report', 'office'], -10.28462348750228), (['<s>', '<s>', 'The', 'deposit', 'office'], -10.656770937877331), (['<s>', '<s>', 'The', 'part', 'office'], -10.6726640630465), (['<s>', '<s>', 'The', 'separate', 'office'], -10.6726640630465)]
[(['<s>', '<s>', 'The', 'department', 'office', 'reports'], -14.565955540401294), (['<s>', '<s>', 'The', 'report', 'office', 'reports'], -14.6040394001

92

## v) Evaluate your context-aware spelling corrector

In [87]:
import random

def replace_characters(sentence, probability):
    modified_sentence = []
    for word in sentence:
        modified_word = ''
        for char in word:
            if char != ' ' and random.random() < probability:
                # Replace non-space character with a visually or acoustically similar character
                # You can customize this part based on your preference or use external libraries for similarity
                modified_char = get_similar_char(char)
                modified_word += modified_char
            else:
                modified_word += char
        modified_sentence.append(modified_word)
    return modified_sentence

def get_similar_char(char):
    # Replace this with your logic to get a visually or acoustically similar character
    # For simplicity, using a basic example here (you can expand this based on your requirements)
    similar_chars = {'a': 'e', 'b': 'd', 'c': 'e', 'd': 'b', 'e': 'a', 'f': 'g',
                     'g': 'f', 'h': 'i', 'i': 'h', 'j': 'k', 'k': 'j', 'l': 'm',
                     'm': 'l', 'n': 'o', 'o': 'n', 'p': 'q', 'q': 'p', 'r': 's',
                     's': 'r', 't': 'u', 'u': 't', 'v': 'w', 'w': 'v', 'x': 'y',
                     'y': 'x', 'z': 'z'}
    return similar_chars.get(char, char)

def modify_corpus(corpus, probability):
    modified_corpus = []
    for sentence in corpus:
        modified_sentence = replace_characters(sentence, probability)
        modified_corpus.append(modified_sentence)
    return modified_corpus

# Example usage with a probability of 0.1 (10% chance of replacing each non-space character)
modified_test_corpus = modify_corpus(test_sents, 0.1)

print_cnt = 0
for sent in modified_test_corpus:
  print_cnt +=1
  print(sent)
  print("______________________")
  if print_cnt == 20:
    break;

['The', 'colmisshon', 'is', 'expacted', 'to', 'approva', 'uhe', 'apqlicathoo', 'at', 'a', 'meeting', 'tomorrow', '.']
______________________
['"', 'The', 'Unhteb', 'Stater', 'and', 'the', 'siy', 'major', 'hndustrial', 'countries', 'are', 'fully', 'committed', 'to', 'implementing', 'nur', 'undertekings', 'hn', 'these', 'afreements', ',"', 'Baker', 'told', 'tie', 'meetings', '.']
______________________
['Soo', 'Line', 'saib', 'in', 'January', 'it', 'vas', 'saejing', 'dids', 'for', 'the', 'qroparty', '.']
______________________
['Cain', '-', 'Sloan', 'has', 'four', 'stnres', 'in', 'Nashville', '.']
______________________
['Romero', 'saib', 'he', 'would', 'tell', 'big', 'duxars', 'nf', 'cnqra', 'meal', 'in', 'London', 'tiat', 'the', 'Philippioes', 'was', 'doing', 'its', 'best', 'to', 'meet', 'EC', 'stanbards', '.']
______________________
['Pretax', 'profius', 'also', 'dipped', 'to', '601', '.', '7', 'lln', 'stg', 'after', '614', '.', '4', 'mln', '.']
______________________
['Asked', 'hn', 

In [1]:
import numpy as np

# Take a portion of the test_corpus and modified_test_corpus
org_sent = cleaned_test_sentences[:5]
wrg_sent = modified_test_corpus[:5]

def correct_corpus_np(corpus, vocab, max_candidates=5):
    corrected_corpus = []
    for sentence in corpus:
        corrected_sentence = beam_search_spelling(sentence, 3,0.2,0.8, generate_candidate_with_distance, score)
        corrected_corpus.append(corrected_sentence)
    return corrected_corpus

corrected_test_corpus = correct_corpus_np(wrg_sent, vocab, 5)

print("Original Test Corpus:")
print(org_sent)

print("\nModified Test Corpus:")
print(wrg_sent)

print("\nCorrected Test Corpus:")
print(corrected_test_corpus)


NameError: name 'test_sents' is not defined

## Evaluate the context-aware spelling corrector in terms of Word Error Rate (WER) and Character Error Rate (CER)

In [100]:
# !pip install evaluate
# !pip install jiwer
from evaluate import load

cor_sent = corrected_test_corpus[:50]

# Flatten the list of lists
flattened_cor_sent = [' '.join(sentence) for sentence in cor_sent]
flattened_org_sent = [' '.join(sentence) for sentence in org_sent]
flattened_wrg_sent =  [' '.join(sentence) for sentence in wrg_sent]
# Transform predictions
predictions1 = [' '.join(flattened_cor_sent)]

references = [' '.join(flattened_org_sent)]

predictions2 = [' '.join(flattened_wrg_sent)]

wer1 = load("wer")  # Load Word-Error-Rate metric
wer_score = wer1.compute(predictions=predictions1, references=references)
print(f"WER score corrected vs original is: {wer_score}")


wer2 = load("wer")  # Load Word-Error-Rate metric
wer_score = wer2.compute(predictions=predictions2, references=references)
print(f"WER score artificial vs original is: {wer_score}")


cer1 = load("cer")
cer_score = cer1.compute(predictions=predictions1, references=references)
print(f"CER score corrected vs original is: {cer_score}")


cer2 = load("cer")
cer_score = cer2.compute(predictions=predictions2, references=references)
print(f"CER score wrong vs original is: {cer_score}")

WER score corrected vs original is: 0.10588235294117647
WER score artificial vs original is: 0.3176470588235294
CER score corrected vs original is: 0.04680851063829787
CER score wrong vs original is: 0.07659574468085106
